In [7]:
import pandas as pd
from datetime import timedelta

数据预处理：
1. 上午时间不变，下午的时间+12
2. 时间里的毫秒去掉
3. 时间读取到pandas内 dates = pd.to_datetime(date_strings, format="%d-%m月 -%y %H.%M.%S.%f")
4. 因为数据只给了同一个月份的，暂且先不考虑跨月和跨年，先考虑跨日
5. 操作内容前缀去掉

In [8]:
class DataProcess:
    def __init__(self, path: str) -> None:
        self.data = pd.read_csv(path)
    
    def process(self):
        self.data = self._data_process(self.data)
        return self.data
    
    @staticmethod
    def _data_process(df: pd.DataFrame):
        df_new = pd.DataFrame()
        data_time_format = "%d-%m月 -%y %H.%M.%S.%f"
        # 拆分出时间和上下午
        df_new['日期时间'] = pd.to_datetime(df['操作时间'].str.rsplit(' ', n=1, expand=True)[0], format=data_time_format)
        # 添加12小时给下午的日期时间
        pm_rows = df['操作时间'].str.endswith("下午")
        time_condition = (df_new['日期时间'].dt.hour != 12)
        df_new.loc[pm_rows & time_condition, '日期时间'] += pd.to_timedelta(12, unit='h')

        df_new['账号'] = df['从账号'].copy()
        df_new['机构'] = df['机构'].copy()
        df_new['操作内容'] = df['操作内容'].str.slice(36, None).str.strip()
        df_new['源IP'] = df['源IP'].copy()
        df_new = df_new.sort_values(by='日期时间', ascending=True)

        df_new.to_csv('data_process.csv')

        return df_new

df = DataProcess('data.csv').process()
df

,日期时间,账号,机构,操作内容,源IP
109868,2023-05-01 08:21:01,29527d9c-1b55-4121-9450-1acc95dbaa62,榆林全市,用户登陆,10.136.71.74
109867,2023-05-01 08:21:02,29527d9c-1b55-4121-9450-1acc95dbaa62,榆林全市,访问首页模块,10.136.71.74
109866,2023-05-01 08:21:08,29527d9c-1b55-4121-9450-1acc95dbaa62,榆林全市,访问5GSA驻留比模块,10.136.71.74
109864,2023-05-01 08:22:54,1c7cc983-5870-4892-af7b-c06a86ce59d7,延安全市,用户登陆,10.136.50.122
109865,2023-05-01 08:22:55,1c7cc983-5870-4892-af7b-c06a86ce59d7,延安全市,访问首页模块,10.136.50.122
...,...,...,...,...,...
5,2023-05-31 23:00:11,dc57ae05-2e96-4866-bffe-501deb9ffb01,产品运营中心,访问首页模块,10.136.14.86
3,2023-05-31 23:00:51,dc57ae05-2e96-4866-bffe-501deb9ffb01,产品运营中心,用户登陆,10.136.14.86
2,2023-05-31 23:00:52,dc57ae05-2e96-4866-bffe-501deb9ffb01,产品运营中心,访问首页模块,10.136.14.86
1,2023-05-31 23:01:00,dc57ae05-2e96-4866-bffe-501deb9ffb01,产品运营中心,访问智能家居质量日报模块,10.136.14.86


分别定义五个类：

业务高频访问：为每一个账户维护一个字典，值为用户访问的业务以及对应的次数和对应的时间。

账号复用：为每一个账号维护一个集合，保存此账号每次登录时的ip和时间，两分钟内如果集合len>=3，即账号复用，每隔2分钟清零。

非常用IP访问：TODO:这个是否要一次读完全部日志？一条一条读无法判断之后还会不会来。

非工作时间访问：不在8:00~19:00之间的日志都是。

登录异常： 为每个账号维护一个字典，记录访问次数，字典值>=8打上登录异常，一分钟清零

         为每个账号维护一个集合，保存此账号登录的ip，一分钟内若集合len>=2视为登录异常

---


清零操作：每次读到一行数据，如果是一分钟清零，如果新数据的分钟数 = 原来数据分钟数+1，将原数据结构清零。

         两分钟清零就是如果读到了新数据分钟数 = min(原始数据分钟数)+2



优化空间：
1. 五个操作共同维护数据结构，{账号1:{访问次数:{}}, 账号2：{}} 
2. 登录异常1分钟2个ip和2分钟三个ip的逻辑能不能合在一起，只用一个字典来处理。
3. 

为了减少损耗，一次读取的时候同时进行五个操作，因此类里的每个detection方法读取的参数是一行Series

✅ 业务高频访问

In [13]:
class HighFrequencyAccess:

    class User:
        __slots__ = ('time', 'business')
        def __init__(self,timestamp):
            self.time=timestamp
            self.business={}

    def __init__(self, max_num) -> None:
        self.max_num = max_num
        self.user_dict = {}

    def detection(self,row) -> bool:
        user_id= row['账号']
        timestamp = row['日期时间']
        transaction  = row['操作内容']
        if user_id in self.user_dict:
            user = self.user_dict[user_id]
            if timestamp-user.time >= timedelta(minutes=1) or timestamp.minute!=user.time.minute:
                del self.user_dict[user_id]
                # 删除之前的数据
                self.user_dict[user_id] = HighFrequencyAccess.User(timestamp)

            user = self.user_dict[user_id]
        else:
            user = HighFrequencyAccess.User(timestamp)
        user.business[transaction] = user.business.get(transaction, 0) + 1
        self.user_dict[user_id] = user
        
        if user.business[transaction]>self.max_num:
            return True
        else:
            return False

309

账号复用

In [7]:
class AccountReuse:
    def __init__(self) -> None:
        self.account_data = {}

    def detection(self, row):
        account = row['账号']
        ip = row['源IP']
        timestamp = row['日期时间']

        # 处理新账户
        if account not in self.account_data:
            self.account_data[account] = {'ips': set(), 'last_timestamp': timestamp}
        
        # 每2分钟清空一个ips集合
        if (timestamp - self.account_data[account]['last_timestamp']) > timedelta(minutes=2):
            self.account_data[account]['ips'] = set()

        self.account_data[account]['ips'].add(ip)
        self.account_data[account]['last_timestamp'] = timestamp

        if len(self.account_data[account]['ips']) >= 3:
            print(f"账号 {account} 在时间 {timestamp} 存在账号复用行为")

In [12]:
account_reuse = AccountReuse()
account_reuse.detection(df)

TypeError: unhashable type: 'Series'

非常用IP访问

In [17]:
class UncommonIp:

    class IpData:
        __slots__ = ('count', 'time')
        def __init__(self) -> None:
            self.count = 0
            self.time = []

    def __init__(self):
        self.account_data = {}

    def dataload(self, row):
        account = row['账号']
        ip = row['源IP']
        timestamp = row['日期时间']
        # 如果字典里没有这个用户，初始化
        if account not in self.account_data:
            self.account_data[account] = {'total':0, 'ips':{}}
        
        self.account_data[account]['total'] += 1
        # 如果用户里没有这个ip，初始化
        if ip not in self.account_data[account]:
            self.account_data[account][ip] = UncommonIp.IpData()
        
        self.account_data[account][ip].count += 1
        # 如果超过count已经超过4就不再统计ip地址了
        if self.account_data[account][ip].count <= 4:
            self.account_data[account][ip].time.append(timestamp)
    
    def detection(self):
        for account, ip_data in self.account_data.items():
            if ip_data['total'] >= 100:
                for key,
                print(account ,ip_data['total'])

u = UncommonIp()
for i,row in df.iterrows():
    u.dataload(row)

u.detection()   

# 315条 >= 100

29527d9c-1b55-4121-9450-1acc95dbaa62 212
d7a106e0-a84e-4203-9606-d86705ef437a 108
7c07ce3b-7dc1-4547-bfa7-259993fcf0e5 329
ba744487-e1f0-46b4-961f-794d9a99a60e 677
f07c37bc-c5c8-4953-beb0-732f70cbe9cd 157
e0eaa2fa-1a44-4741-8154-2efdaeffa47d 142
59e92611-89bb-47bb-8eb2-19fe4b37573b 134
5334796c-c039-4f10-b1fe-56b29f3deb19 175
efe7b7ba-42bc-4272-b049-638b7c022f07 309
e655a174-6c8d-4dbb-844f-db16518ecb9f 128
a818985c-9381-48c1-8d0f-32d46ca7bdf2 1112
e93edf41-2887-4fcd-805f-680dfc174f5a 254
e9660eaf-81f2-4fd2-be5c-d563c52d5124 407
62e2d332-bebe-42de-b62a-7aa5037dce9f 374
404605c3-18d7-45da-becc-3ca40d31b15c 891
8a96a904-e6be-4aaf-a814-e612326f5639 141
ca71ec3a-359e-4e5d-80c2-d4373034fdae 145
9de6de98-c8c2-4534-9b9e-d08d03b9deee 1368
3b61d90c-2ff5-4e9a-8959-815c76bb03c9 675
65b7a7fd-2bf1-4ff3-bf89-3a6b0d79897d 345
fb49e7e7-4f98-43d1-aa72-054fcc5c93a5 333
73006ec3-370f-42f2-8ed7-27d3e5b0f8a6 771
a19aaf53-3d48-406f-9c5a-11b875b7daa0 518
5f311db5-04ac-4bf5-b128-f9be49814210 108
3c3e0427-15e3-

✅ 非工作时间访问

In [51]:
class OutOfHoursAccess:
    def __init__(self, start, end) -> None:
        self.start = start
        self.end = end

    def detection(self, row: pd.core.series.Series) -> bool:
        datetime_value = row['日期时间'].time()
        # TODO: 19:00:00算不算？
        if not (self.start <= datetime_value <= self.end):
            # print(f"Row {row}: {datetime_value} is not in the range.")
            return True
        return False


登录异常

In [ ]:
class LoginException:
    def __init__(self) -> None:
        pass

    def detection(self) -> int:
        pass

main函数

In [ ]:
# 1. 业务高频访问
max_num = 5
highfrequency = HighFrequencyAccess(max_num)
# 2. 账户复用
accountreuse = AccountReuse()
# 3. 非常用IP
uncommonip = UncommonIp()
# 4. 非工作时间
start = pd.to_datetime('08:00:00').time()
end = pd.to_datetime('19:00:00').time()
outHour = OutOfHoursAccess(start, end)
# 5. 登录异常
login = LoginException()

for i, row in df.iterrows():
    outHour.detection(row)